# Draft analysis 

---

Group name: Gruppe D

---


## 1. Introduction


### 1.1 Motivation
Die Untersuchung beschäftigt sich mit dem Thema Mobilität, insbesondere im Hinblick auf die Pünktlichkeit des öffentlichen Personenverkehrs (Gäbel, 2021). Dieser umfasst die Grundversorgung von Mobilität auf Straße, Schiene, Wasser und mittels Luftseilbahn. Das Thema Mobilität ist aufgrund  aktueller gesellschaftlicher und politischer Problemstellungen rund um die Erreichung von Klimazielen sehr relevant (Stückelberger, 2021). Als “Welthauptstadt des ÖPNV” beschreibt Kühne (2018) die schweizer Stadt Zürich. Er verdeutlicht, dass 41% der Stadtbewohner:innen den öffentlichen Nahverkehr benutzen. Es liegt eine Umfangreiche Datensammlung der Verkehrsbezüge Zürich vor, mit der das bestehende Verkersangebot untersucht wird. Hierbei werden ausschließlich Daten von Fahrzeugen mit festen Soll-Abfahrtszeiten berücksichtigt. Ziel der Untersuchung sind Handlungsempfehlungen zur Optimierung des bestehenden Verkehrsangebots.

**Literaturverzeichnis**

Gäbel, L., & Reinhold, T. (2021). Wie Bahn und Bus schneller und  pünktlicher werden. Der Nahverkehr, 39(4), 52.

Schäfer, R. (2021, June 30). Wien - so macht Nahverkehr Spaß (season 1, episode 6). WDR Online. https://open.spotify.com/episode/6VKmnQYigspu4CQeP2wH4P?si=43abefe2d3fa4ad4

Stückelberger, U. (2021). Klimaziele: Der öffentliche Verkehr ist Teil der Lösung! Der Nahverkehr, 39(4), 3. https://www.wiso-net.de/document/DNV__341c9126826bbe49155cb41d63369c6d78cc80b3


### 1.2 Data
Zur Analyse der Daten stehen wöchentliche csv-Dateien zur Verfügung, welche eine Vielzahl an Daten beinhalten. Jeder Eintrag stellt einen geplanten Halt eines Fahrzeugs der Verkehrsbetriebe Zürich dar und gibt eine präzise Auskunft über Datum, Linie, Fahrzeug, Haltestelle und vielem mehr. Somit ermöglicht der Datensatz einen Vergleich zwischen den geplanten und tatsächlich erfolgten Fahrzeiten der Fahrzeuge der Verkehrsbetriebe Zürich. Da vielen Daten aus der Tabelle eine interne ID und somit einen Fremdschlüssel zu anderen Tabellen darstellen, sind weitere Tabellen vorhanden, um diese Daten mit konkreten, verständlichen Bezeichnungen verknüpfen zu können.

Der Datenlieferant ist Verkehrsbetriebe Zürich, Departement der Industriellen Betriebe. Die Daten wurden erstmals 2015 erfasst und werden seither wöchentlich sonntags aktualisiert. Sämtliche Zeiten werden dabei auf die Sekunde gemessen. Über die genaue Bereitstellung der Daten gibt es keine Informationen. Da die Daten wöchentlich aktualisiert werden, ist davon auszugehen, dass die Daten durch einen automatischen Export aus einer oder mehreren weiteren Datenbanktabellen zusammengetragen und miteinander verknüpft werden.

Für die Untersuchung werden Daten vom September 2022 betrachtet.

### 1.3 Research Question
Aus der herangegangenen Motiation und Darlegung der Daten stellt sich die Frage: Wie kann der bestehnde Nahverkehr in Zürich verbessert werden? 

#### 1.3.1 Hypothesen
1. Die meisten Verspätungen finden im Berufsverkehr (morgens ca. 6:30 Uhr bis 08:30 Uhr und abends ca. 16:00-18:00 Uhr) statt
2. Es gibt mehr Verspätungen unter der Woche als am Wochenende
3. Hauptverkehrslinien innerhalb der Stadtmitte sind von den Verspätungen stärker betroffen als andere

### 1.4 Data Dictionary

## 2. Setup

In [1]:
import pandas as pd
import altair as alt

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
alt.data_transformers.disable_max_rows()

# reset display of max rows to enable displaying all columns of large dfs
pd.options.display.max_columns = None

## 3. Data

### 3.1 Import data

Import der benötigten CSV-Dateien

In [2]:
# import csv's

# 1. Septemberwoche 2022
df_week1 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220904_20220910.csv")

# 2. Septemberwoche 2022
df_week2 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220911_20220917.csv")

# 3. Septemberwoche 2022
df_week3 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220918_20220924.csv")

# 4. Septemberwoche 20222
df_week4 = pd.read_csv("https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Fahrzeiten_SOLL_IST_20220925_20221001.csv")

# Haltestellen
df_haltestellen = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd/download/Haltestelle.csv')

# Linien
df_linie = pd.read_csv('https://data.stadt-zuerich.ch/dataset/vbz_fahrgastzahlen_ogd/download/LINIE.csv', delimiter=';')

In [3]:
# concat dfs
df_tot = pd.concat([df_week1, df_week2, df_week3, df_week4], ignore_index=True)

# control concat
print("Daten erfolgreich zusammengeführt") if len(df_tot.index) == len(df_week1.index) + len(df_week2.index) + len(df_week3.index) + len(df_week4.index) else print("Fehler beim Zusammenführen der Daten")

Daten erfolgreich zusammengeführt


Übersicht des DataFrames 

In [4]:
df_tot.head()

,linie,richtung,betriebsdatum,fahrzeug,kurs,seq_von,halt_diva_von,halt_punkt_diva_von,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,seq_nach,halt_diva_nach,halt_punkt_diva_nach,halt_kurz_nach1,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,fahrt_id,fahrweg_id,fw_no,fw_typ,fw_kurz,fw_lang,umlauf_von,halt_id_von,halt_id_nach,halt_punkt_id_von,halt_punkt_id_nach
0,2,1,08.09.22,3059,1,25,440,0,BELL,08.09.22,30618,30603,30636,30632,26,1845,0,OPER,08.09.22,30702,30713,30732,30740,42241,173050,1,1,1,SGEI - BTIE,284307,1565,2104,52890,10538
1,2,1,08.09.22,2069,4,25,440,0,BELL,08.09.22,26118,26053,26136,26069,26,1845,0,OPER,08.09.22,26202,26139,26232,26158,42313,173050,1,1,1,SGEI - BTIE,284477,1565,2104,52890,10538
2,2,1,08.09.22,3059,1,25,440,0,BELL,08.09.22,36462,36409,36486,36442,26,1845,0,OPER,08.09.22,36564,36528,36588,36553,42243,173050,1,1,1,SGEI - BTIE,284307,1565,2104,52890,10538
3,2,1,08.09.22,2079,10,25,440,0,BELL,08.09.22,34662,34656,34686,34675,26,1845,0,OPER,08.09.22,34764,34770,34788,34787,42471,173050,1,1,1,SGEI - BTIE,284540,1565,2104,52890,10538
4,2,1,08.09.22,2066,2,25,440,0,BELL,08.09.22,31068,31100,31086,31121,26,1845,0,OPER,08.09.22,31152,31220,31182,31266,42261,173050,1,1,1,SGEI - BTIE,284361,1565,2104,52890,10538


### 3.2 Data structure

In [5]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5682894 entries, 0 to 5682893
Data columns (total 34 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   linie                 int64 
 1   richtung              int64 
 2   betriebsdatum         object
 3   fahrzeug              int64 
 4   kurs                  int64 
 5   seq_von               int64 
 6   halt_diva_von         int64 
 7   halt_punkt_diva_von   int64 
 8   halt_kurz_von1        object
 9   datum_von             object
 10  soll_an_von           int64 
 11  ist_an_von            int64 
 12  soll_ab_von           int64 
 13  ist_ab_von            int64 
 14  seq_nach              int64 
 15  halt_diva_nach        int64 
 16  halt_punkt_diva_nach  int64 
 17  halt_kurz_nach1       object
 18  datum_nach            object
 19  soll_an_nach          int64 
 20  ist_an_nach1          int64 
 21  soll_ab_nach          int64 
 22  ist_ab_nach           int64 
 23  fahrt_id              int64 
 24

### 3.3 Data corrections

In [6]:
# remove unnecessary columns in df_tot
df_tot = df_tot.drop(['kurs', 'seq_von', 'halt_diva_von', 'halt_punkt_diva_von', 'seq_nach',
'halt_diva_nach', 'halt_punkt_diva_nach', 'halt_kurz_nach1', 'fahrt_id', 'fahrweg_id', 'fw_no', 'fw_typ',
'fw_kurz', 'fw_lang', 'umlauf_von', 'halt_punkt_id_von', 'halt_punkt_id_nach'], axis=1)


In [7]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5682894 entries, 0 to 5682893
Data columns (total 17 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   linie           int64 
 1   richtung        int64 
 2   betriebsdatum   object
 3   fahrzeug        int64 
 4   halt_kurz_von1  object
 5   datum_von       object
 6   soll_an_von     int64 
 7   ist_an_von      int64 
 8   soll_ab_von     int64 
 9   ist_ab_von      int64 
 10  datum_nach      object
 11  soll_an_nach    int64 
 12  ist_an_nach1    int64 
 13  soll_ab_nach    int64 
 14  ist_ab_nach     int64 
 15  halt_id_von     int64 
 16  halt_id_nach    int64 
dtypes: int64(13), object(4)
memory usage: 737.1+ MB


In [8]:
df_haltestellen.head()

,halt_id,halt_diva,halt_kurz,halt_lang,halt_ist_aktiv
0,143,2570,BirWSL,"Birmensdorf ZH, Sternen/WSL",True
1,309,3356,WalBir,"Waldegg, Birmensdorferstrasse",True
2,373,6232,FRAF07,"Zürich Flughafen, Fracht",True
3,539,2655,TBAH01,"Thalwil, Bahnhof",True
4,588,3027,FLUG07,"Zürich Flughafen, Bahnhof",True


In [9]:
# remove unnecessary columns in df_haltestellen
df_haltestellen = df_haltestellen.drop(['halt_diva', 'halt_kurz', 'halt_ist_aktiv'], axis=1)
df_haltestellen.head()

,halt_id,halt_lang
0,143,"Birmensdorf ZH, Sternen/WSL"
1,309,"Waldegg, Birmensdorferstrasse"
2,373,"Zürich Flughafen, Fracht"
3,539,"Thalwil, Bahnhof"
4,588,"Zürich Flughafen, Bahnhof"


In [10]:
# join both dataframes
df_tot = df_tot.join(df_haltestellen.set_index('halt_id'), on='halt_id_von')
df_tot = df_tot.rename(columns={"halt_lang": "halt_lang_von"})

df_tot = df_tot.join(df_haltestellen.set_index('halt_id'), on='halt_id_nach')
df_tot = df_tot.rename(columns={"halt_lang": "halt_lang_nach"})

df_tot.head()

,linie,richtung,betriebsdatum,fahrzeug,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,halt_id_von,halt_id_nach,halt_lang_von,halt_lang_nach
0,2,1,08.09.22,3059,BELL,08.09.22,30618,30603,30636,30632,08.09.22,30702,30713,30732,30740,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus"
1,2,1,08.09.22,2069,BELL,08.09.22,26118,26053,26136,26069,08.09.22,26202,26139,26232,26158,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus"
2,2,1,08.09.22,3059,BELL,08.09.22,36462,36409,36486,36442,08.09.22,36564,36528,36588,36553,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus"
3,2,1,08.09.22,2079,BELL,08.09.22,34662,34656,34686,34675,08.09.22,34764,34770,34788,34787,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus"
4,2,1,08.09.22,2066,BELL,08.09.22,31068,31100,31086,31121,08.09.22,31152,31220,31182,31266,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus"


In [11]:
# remove unnecessary columns in df_linie
df_linie = df_linie.drop(['Linien_Id'], axis=1)

# join to dataframes
df_tot = df_tot.join(df_linie.set_index('Linienname'), on='linie')

In [12]:
# calculate the punctuality per line
df_tot['pünktlichkeit'] = df_tot.apply(lambda x: "zu spät" if x["ist_an_nach1"] - x["soll_an_nach"] >= 120 else "zu früh"
                                            if x["ist_ab_nach"] - x["soll_ab_nach"]<= -60 else "pünktlich", axis=1)

# calculate time difference
df_tot["zeit_differenz"] = df_tot.apply(lambda x: x["ist_an_nach1"] - x["soll_an_nach"] if x["pünktlichkeit"] == "zu spät"
                                            else x["ist_ab_nach"] - x["soll_ab_nach"] if x["pünktlichkeit"] == "zu früh"
                                            else 0, axis=1)

In [13]:
# add columns with day of week (0 = monday; 6 = sunday)
# dayfirst=True parameter is necessary to not switch day and month
df_tot["wochentag_von"] = pd.to_datetime(df_tot["datum_von"], dayfirst=True).dt.day_of_week
df_tot["wochentag_nach"] = pd.to_datetime(df_tot["datum_nach"], dayfirst=True).dt.day_of_week

# add columns with name of weekdays
df_tot["wochentag_name_von"] = df_tot.apply(lambda x: "Montag" if x["wochentag_von"] == 0
                                                else "Dienstag" if x["wochentag_von"] == 1
                                                else "Mittwoch" if x["wochentag_von"] == 2
                                                else "Donnerstag" if x["wochentag_von"] == 3
                                                else "Freitag" if x["wochentag_von"] == 4
                                                else "Samstag" if x["wochentag_von"] == 5
                                                else "Sonntag", axis=1)
df_tot["wochentag_name_nach"] = df_tot.apply(lambda x: "Montag" if x["wochentag_nach"] == 0
                                                else "Dienstag" if x["wochentag_nach"] == 1
                                                else "Mittwoch" if x["wochentag_nach"] == 2
                                                else "Donnerstag" if x["wochentag_nach"] == 3
                                                else "Freitag" if x["wochentag_nach"] == 4
                                                else "Samstag" if x["wochentag_nach"] == 5
                                                else "Sonntag", axis=1)

# add column with calendar week
df_tot["kw_von"] = pd.to_datetime(df_tot["datum_von"], dayfirst=True).dt.isocalendar().week
df_tot["kw_nach"] = pd.to_datetime(df_tot["datum_nach"], dayfirst=True).dt.isocalendar().week
                                                

In [14]:
df_tot.head()

,linie,richtung,betriebsdatum,fahrzeug,halt_kurz_von1,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,halt_id_von,halt_id_nach,halt_lang_von,halt_lang_nach,VSYS,Linienname_Fahrgastauskunft,pünktlichkeit,zeit_differenz,wochentag_von,wochentag_nach,wochentag_name_von,wochentag_name_nach,kw_von,kw_nach
0,2,1,08.09.22,3059,BELL,08.09.22,30618,30603,30636,30632,08.09.22,30702,30713,30732,30740,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36
1,2,1,08.09.22,2069,BELL,08.09.22,26118,26053,26136,26069,08.09.22,26202,26139,26232,26158,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus",T,2,zu früh,-74,3,3,Donnerstag,Donnerstag,36,36
2,2,1,08.09.22,3059,BELL,08.09.22,36462,36409,36486,36442,08.09.22,36564,36528,36588,36553,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36
3,2,1,08.09.22,2079,BELL,08.09.22,34662,34656,34686,34675,08.09.22,34764,34770,34788,34787,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36
4,2,1,08.09.22,2066,BELL,08.09.22,31068,31100,31086,31121,08.09.22,31152,31220,31182,31266,1565,2104,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36


In [15]:
df_tot = df_tot.drop(['fahrzeug', 'halt_kurz_von1', 'halt_id_von', 'halt_id_nach'], axis=1)

df_tot.head()

,linie,richtung,betriebsdatum,datum_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,datum_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach,halt_lang_von,halt_lang_nach,VSYS,Linienname_Fahrgastauskunft,pünktlichkeit,zeit_differenz,wochentag_von,wochentag_nach,wochentag_name_von,wochentag_name_nach,kw_von,kw_nach
0,2,1,08.09.22,08.09.22,30618,30603,30636,30632,08.09.22,30702,30713,30732,30740,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36
1,2,1,08.09.22,08.09.22,26118,26053,26136,26069,08.09.22,26202,26139,26232,26158,"Zürich, Bellevue","Zürich, Opernhaus",T,2,zu früh,-74,3,3,Donnerstag,Donnerstag,36,36
2,2,1,08.09.22,08.09.22,36462,36409,36486,36442,08.09.22,36564,36528,36588,36553,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36
3,2,1,08.09.22,08.09.22,34662,34656,34686,34675,08.09.22,34764,34770,34788,34787,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36
4,2,1,08.09.22,08.09.22,31068,31100,31086,31121,08.09.22,31152,31220,31182,31266,"Zürich, Bellevue","Zürich, Opernhaus",T,2,pünktlich,0,3,3,Donnerstag,Donnerstag,36,36


In [16]:
# Spaltensortieren
df_tot = df_tot.loc[:,['richtung', 'linie', 'Linienname_Fahrgastauskunft', 'VSYS', 'betriebsdatum', 'datum_von', 'kw_von', 'wochentag_von', 'wochentag_name_von', 'datum_nach', 'kw_nach', 'wochentag_nach', 'wochentag_name_nach', 'pünktlichkeit', 'zeit_differenz', 'halt_lang_von', 'soll_an_von', 'ist_an_von', 'soll_ab_von', 'ist_ab_von', 'halt_lang_nach', 'soll_an_nach', 'ist_an_nach1', 'soll_ab_nach', 'ist_ab_nach']]

df_tot.head()

,richtung,linie,Linienname_Fahrgastauskunft,VSYS,betriebsdatum,datum_von,kw_von,wochentag_von,wochentag_name_von,datum_nach,kw_nach,wochentag_nach,wochentag_name_nach,pünktlichkeit,zeit_differenz,halt_lang_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,halt_lang_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach
0,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",30618,30603,30636,30632,"Zürich, Opernhaus",30702,30713,30732,30740
1,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,zu früh,-74,"Zürich, Bellevue",26118,26053,26136,26069,"Zürich, Opernhaus",26202,26139,26232,26158
2,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",36462,36409,36486,36442,"Zürich, Opernhaus",36564,36528,36588,36553
3,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",34662,34656,34686,34675,"Zürich, Opernhaus",34764,34770,34788,34787
4,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",31068,31100,31086,31121,"Zürich, Opernhaus",31152,31220,31182,31266


In [17]:
# rename columns to improve readability
df_tot = df_tot.rename(columns={"Linienname_Fahrgastauskunft": "linie_fahrgastauskunft", 'VSYS': 'verkehrssystem', 'halt_lang_von': 'halt_von', 'halt_lang_nach': 'halt_nach', })

df_tot.head()

,richtung,linie,linie_fahrgastauskunft,verkehrssystem,betriebsdatum,datum_von,kw_von,wochentag_von,wochentag_name_von,datum_nach,kw_nach,wochentag_nach,wochentag_name_nach,pünktlichkeit,zeit_differenz,halt_von,soll_an_von,ist_an_von,soll_ab_von,ist_ab_von,halt_nach,soll_an_nach,ist_an_nach1,soll_ab_nach,ist_ab_nach
0,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",30618,30603,30636,30632,"Zürich, Opernhaus",30702,30713,30732,30740
1,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,zu früh,-74,"Zürich, Bellevue",26118,26053,26136,26069,"Zürich, Opernhaus",26202,26139,26232,26158
2,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",36462,36409,36486,36442,"Zürich, Opernhaus",36564,36528,36588,36553
3,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",34662,34656,34686,34675,"Zürich, Opernhaus",34764,34770,34788,34787
4,1,2,2,T,08.09.22,08.09.22,36,3,Donnerstag,08.09.22,36,3,Donnerstag,pünktlich,0,"Zürich, Bellevue",31068,31100,31086,31121,"Zürich, Opernhaus",31152,31220,31182,31266


In [18]:
df_tot.dtypes

richtung                   int64
linie                      int64
linie_fahrgastauskunft    object
verkehrssystem            object
betriebsdatum             object
datum_von                 object
kw_von                    UInt32
wochentag_von              int64
wochentag_name_von        object
datum_nach                object
kw_nach                   UInt32
wochentag_nach             int64
wochentag_name_nach       object
pünktlichkeit             object
zeit_differenz             int64
halt_von                  object
soll_an_von                int64
ist_an_von                 int64
soll_ab_von                int64
ist_ab_von                 int64
halt_nach                 object
soll_an_nach               int64
ist_an_nach1               int64
soll_ab_nach               int64
ist_ab_nach                int64
dtype: object

## 4. Analysis

### 4.1 Descriptive statistics

In [19]:
# row counts
row_count_tot = len(df_tot.index)
row_count_early = len(df_tot[df_tot["pünktlichkeit"] == "zu früh"].index)
row_count_punctual = len(df_tot[df_tot["pünktlichkeit"] == "pünktlich"].index)
row_count_late = len(df_tot[df_tot["pünktlichkeit"] == "zu spät"].index)

# create lambda function to add decimal marks
add_decimal = lambda x: format(x, ",d")

print("row count total: ", add_decimal(row_count_tot))
print(f"row count 'zu früh': {add_decimal(row_count_early)} ({round((row_count_early/row_count_tot)*100, 2)}%)")
print(f"row count 'pünktlich': {add_decimal(row_count_punctual)} ({round((row_count_punctual/row_count_tot)*100, 2)}%)")
print(f"row count 'zu spät': {add_decimal(row_count_late)} ({round((row_count_late/row_count_tot)*100, 2)}%)")

row count total:  5,682,894
row count 'zu früh': 122,750 (2.16%)
row count 'pünktlich': 4,750,617 (83.6%)
row count 'zu spät': 809,527 (14.24%)


Wie zu sehen ist, sind etwa 14,2% der Fahrzeuge von einer Verspätung betroffen. Dieser Wert ist nicht gerade wenig, eine Analyse lohnt sich dementsprechend, um Optimierungspotenziale feststellen zu können.

In [20]:

# look at time difference values where pünktlichkeit is "zu früh"
print("Zeit-Differenz bei Verfrühung")
df_tot['zeit_differenz'].where(df_tot["pünktlichkeit"] == "zu früh").describe()


Zeit-Differenz bei Verfrühung


count    122750.000000
mean       -111.936293
std         246.459636
min       -5400.000000
25%         -83.000000
50%         -70.000000
75%         -64.000000
max         -60.000000
Name: zeit_differenz, dtype: float64

In [21]:
# look at time difference values where pünktlichkeit is "zu spät"
print("Zeit-Differenz bei Verspätung")
df_tot['zeit_differenz'].where(df_tot["pünktlichkeit"] == "zu spät").describe()

Zeit-Differenz bei Verspätung


count    809527.000000
mean        249.278298
std         228.435504
min         120.000000
25%         144.000000
50%         182.000000
75%         261.000000
max        6990.000000
Name: zeit_differenz, dtype: float64

In [22]:
freq = df_tot.groupby(['linie', 'pünktlichkeit']).size() 

freq

linie  pünktlichkeit
2      pünktlich        202365
       zu früh            7340
       zu spät           19531
3      pünktlich        134774
       zu früh            3498
                         ...  
918    zu früh              13
       zu spät             163
919    pünktlich         22282
       zu früh             184
       zu spät            1468
Length: 216, dtype: int64

In [23]:
df_week = df_tot.where(df_tot["kw_von"] == "36")

chart = alt.Chart(df_week).mark_bar().encode(
    column = alt.Column('pünktlichkeit'),
    x=alt.X('linie'),
    y=alt.Y('zeit_differenz')
)

chart.display()

In [ ]:
# weitere Werte für Deskriptive Analyse:
# Modalwert bzw. Modus
# Histogramme für Linie, Station, Tag


### 4.2 Exploratory data analysis

In [ ]:
# Bestandteile: -> in den Daten verborgene Muster, Strukturen, Auffälligkeiten identifizieren
# Boxplots/Diagramme für einzelne Linie(n)
# Boxplots/Diagramme für einzelne Station(en)
# Boxplots/Diagramme für Stunden/Tage/Wochen

## 5. Visualizations

### 5.1 Visualization ideas

### 5.2 Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## 6. Conclusion and recommended action